# ロバストMDP

参考
* [Robust dynamic programming](http://www.corc.ieor.columbia.edu/reports/techreports/tr-2002-07.pdf)
* [バナッハの不動点定理](https://mathlandscape.com/banach-fixed-pt/)
* [Robust Control of Markov Decision Processes with Uncertain Transition Matrices](https://people.eecs.berkeley.edu/~elghaoui/Pubs/RobMDP_OR2005.pdf)：完全双対性の証明など．こっちのほうがわかりやすいかも？
* [First-order Policy Optimization for Robust Markov Decision Process](https://arxiv.org/abs/2209.10579)
    * Remark 1.1でなぜ遷移のUncertainty setを考えるのか？みたいな話をしていて良い

シミュレータを使ったアプリケーションではしばしば，シミュレータ上の遷移確率$P^o$で方策を学習させてから実世界にデプロイすることを考えます．
しかし，実世界の遷移確率は$P^o$に対して必ずズレがあります．
ロバストMDPでは，$P^o$からのズレを含めた遷移確率の集合を$\mathcal{P}$として，$\mathcal{P}$の中の最悪ケースに対してもある程度の性能を保証するように学習することを目標にします．


## ロバストMDPによって性能に差が出る例

参考：
* [First-order Policy Optimization for Robust Markov Decision Process](https://arxiv.org/abs/2209.10579)

ロバストMDPはいつ重要になるでしょうか？次のようなケースを考えてみましょう：

![Robust-MDP](figs/SA-Robust-MDP.png)

**(a)のMDP**

* 状態$S_0$では２つの行動$\{L, R\}$があります．$R$だと報酬$+1$，$L$だと報酬$+0$が発生します．
* また，状態$S_{k-1, L}$から$S_{k, L}$の部分で報酬$+(1+\epsilon) \gamma^{-k+1}$が発生します．ここで$\epsilon \ll 1$です．例えば$\epsilon=0.01$など．
    * つまり，$S_0$においてLを選択すると$1+\epsilon$のリターン，右に行くと$+1$のリターンが発生します．よって左に行くのが最適解です．

**(b)のMDP**

* $\mathbb{P}_u\left(S_{m+1, L} \mid S_{m, L}\right)=p, \mathbb{P}_u\left(S_{m, R} \mid S_{m, L}\right)=1-p$とします．すなわち，(a)のMDPと遷移確率がほとんど変わりません
* $S_{i, R}$に到達するとずっとその場に留まります．
* このとき，左に行くとリターンは$1+\epsilon p^{k-1}$であり，これは$k\to \infty$でリターンが$0$になります．よって，このMDPでは$S_0$で右に行くのが最適解です．

(a)と(b)のMDPでは遷移が少し違うだけですが，その最適方策と得られるリターンが全然違います．
また，左右等確率で選択するのは良い方策にはならず，良さそうな解は右を選択する方策です．
右を選択する方策は$\mathcal{M}$では$\epsilon$-最適方策，そして$\mathcal{M}_u$では最適方策になります．


## ロバスト動的計画法

ロバストMDPの基本的な解き方の一つに，ロバスト動的計画法があります．

**表記**

* 更新の回数：$K$
* エフェクティブホライゾン：$H:=1 / (1 - \gamma)$
* 遷移確率のUncertainty set： $ \mathcal{P}=\otimes \mathcal{P}_{s, a}, \text { where, } \mathcal{P}_{s, a}=\left\{P_{s, a} \in[0,1]^{|\mathcal{S}|}: D\left(P_{s, a}, P_{s, a}^o\right) \leq c_r, \sum_{s^{\prime} \in \mathcal{S}} P_{s, a}\left(s^{\prime}\right)=1\right\}, $
    * ここで$D$は分布間の距離です。今回は特にf-ダイバージェンス$D_f(P \| Q)=\sum_x Q(x) f\left(\frac{P(x)}{Q(x)}\right)$を考えます。
* 遷移確率$P$での価値関数：$V_{\pi, P}(s)=\mathbb{E}_{\pi, P}\left[\sum_{t=0}^{\infty} \gamma^t r\left(s_t, a_t\right) \mid s_0=s\right]$
* $P$における最適価値関数と最適方策：$V_P^*=\max _\pi V_{\pi, P}, \quad \pi_P^*=\underset{\pi}{\arg \max } V_{\pi, P}$
* ロバストな価値関数と最適価値関数：$V^\pi=\inf _{P \in \mathcal{P}} V_{\pi, P}, \quad V^*=\sup _\pi \inf _{P \in \mathcal{P}} V_{\pi, P}$

---

**定理：マルコフ最適性**

方策集合が全ての方策を含むとき、ロバストな最適価値関数 $V^*=\sup_{\pi} \inf _{P \in \mathcal{P}} V_{\pi, P}$を与える$\pi$は決定的方策になる。

これの証明は通常のマルコフ決定過程とほぼ同じらしい。Puterman(1994)を見よう。

---

---

**完全双対性**

$$
V^*=\sup _\pi \inf _{P \in \mathcal{P}} V_{\pi, P}
=\inf _{P \in \mathcal{P}} \sup _\pi V_{\pi, P}
$$

が成立．
証明は[RL_robust_MDP_zero_duality.ipynb](RL_robust_MDP_zero_duality.ipynb)

---

---

**ベルマン作用素**

* 任意のベクトル$v$と集合$\mathcal{B}$について、$\sigma_{\mathcal{B}}(v)=\inf \left\{u^{\top} v: u \in \mathcal{B}\right\}$とする。
* ロバストベルマン最適作用素：$T(V)(s)=\max _a(r(s, a)+ + \gamma \sigma_{\mathcal{P}_{s, a}}(V))$

ロバストベルマン最適作用素はcontractionになっている。つまり、任意の$U, V \in \mathcal{V}$について、

$$\|T U-T V\| \leq \gamma\|U-V\|$$

であり、また、$T$は唯一の解をもち、それは$V^*$である。

前半部分をまず証明します。$TU \geq TV$を仮定します。このとき、次を満たすように決定的方策$d$を選びます。

$$r\left(s, d(s)\right)+\gamma \inf _{p \in \mathcal{P}(s, d(s))} \mathbf{E}^p\left[U\left(s^{\prime}\right)\right] \geq T U(s)-\epsilon$$

続いて、次を満たすように$p_s \in \mathcal{P}(s, d(s))$を選びます。

$$r\left(s, d(s)\right)+\gamma \mathbf{E}^{p_s}\left[V\left(s^{\prime}\right)\right] \leq r\left(s, d(s)\right)+\lambda \inf _{p \in \mathcal{P}(s, d(s))} \mathbf{E}^p\left[V\left(s^{\prime}\right)\right]+\epsilon$$

これを使って変形すると、

$$
\begin{aligned}
0 \leq T U(s)- T V(s) & \leq\left(r\left(s, d(s)\right)+\gamma \inf _{p \in \mathcal{P}(s, d(s))} \mathbf{E}^p\left[U\left(s^{\prime}\right)\right]+\epsilon\right)- \left(r\left(s, d(s)\right)+\gamma\inf _{p \in \mathcal{P}(s, d(s))} \mathbf{E}^p\left[V\left(s^{\prime}\right)\right]\right), \\
& \leq \left(r\left(s, d(s)\right)+\gamma \mathbf{E}^{p_s}\left[U\left(s^{\prime}\right)\right]+\epsilon\right)-\left(r\left(s, d(s)\right)+\gamma \mathbf{E}^{p_s}\left[V\left(s^{\prime}\right)\right]-\epsilon\right), \\
& =\gamma \mathbf{E}^{p_s}[U-V]+2 \epsilon, \\
& \leq \gamma \mathbf{E}^{p_s}|U-V|+2 \epsilon, \\
& \leq \gamma\|U-V\|+2 \epsilon .
\end{aligned}
$$

これでcontractionが示せました。唯一の収束点の方は省略しますが、[バナッハの不動点定理](https://mathlandscape.com/banach-fixed-pt/)から不動点の存在は示せます。詳細は[Robust dynamic programming](http://www.corc.ieor.columbia.edu/reports/techreports/tr-2002-07.pdf)参照。

---


### $\sigma_{\mathcal{P}_{s, a}}(V)$について

ロバスト動的計画法では、$\sigma_{\mathcal{P}_{s, a}}(v)=\inf \left\{p^{\top} v: p \in \mathcal{P}_{s, a}\right\}$をとく必要があります。これは単純に制約付き最適化を実行すると非常に遅いですが、特定の制約の場合はラグランジュの未定乗数法を使えば効率的に解くことができます。　


**$D$がKLダイバージェンスのとき**

$D\left(p_1 \| p_2\right)=\sum_{s \in \mathcal{S}} p_1(s) \log \left(\frac{p_1(s)}{p_2(s)}\right)$の場合を考えましょう。

定義より$\sigma_{\mathcal{P}_{s, a}}(v)$は$t > 0$について
$$
\begin{array}{ll}
\text { minimize }_p & p^{\top} v\\
\text { subject to } & p \in \mathcal{P}=\{p \in [0, 1]^{|\mathcal{S}|}: D(p \| q) \leq t, \sum_{s\in \mathcal{S}}p(s)=1\}
\end{array}
$$
でした（$t$は$c_\tau$と、$q$は$P_{s, a}^o$と読み替えてください）。
この解は

$$
-\min _{\lambda \geq 0}t \lambda+\lambda \log \left(q^T\exp \left(-\frac{v}{\lambda}\right)\right)
$$

と等価です。

簡単に示してみましょう。まず、制約付き最適化問題のラグランジアンは$\lambda \geq 0$と$\mu \in \mathbb{R}$について、

$$\mathcal{L}=\sum_{s \in \mathcal{S}} p(s) v(s)-\lambda\left(t-\sum_{s \in \mathcal{S}} p(s) \log \left(\frac{p(s)}{q(s)}\right)\right)-\mu\left(\sum_{s \in \mathcal{S}} p(s)-1\right)$$

です。これを$p(s)$について微分すれば、

$$
\lambda \log \left(\frac{p(s)}{q(s)}\right)+v(s)=\mu-\lambda
$$

を得ます。これを$p(s)$について解けば

$$p(s)=q(s) \exp \left(-1+\frac{\mu-v(s)}{\lambda}\right)$$

です。
$\sum_{s \in \mathcal{S}} p(s)=1$を使ってラグランジアンを変形すると

$$\mathcal{L}=\sum_{s \in \mathcal{S}} p(s) \left(v(s) + \lambda \log \left(\frac{p(s)}{q(s)}\right) \right)-\lambda t
=\mu - \lambda -\lambda t
$$

です。
また、$p(s)=q(s) \exp \left(-1+\frac{\mu-v(s)}{\lambda}\right)$と$\sum_{s \in \mathcal{S}} p(s)=1$を合わせると、

$$\exp\left(\frac{\mu-\lambda}{\lambda}\right)\sum_{s \in \mathcal{S}} q(s) \exp \left(\frac{-v(s)}{\lambda}\right) = 1$$
より、

$$\mu-\lambda = -\lambda \log \left(\sum_{s \in \mathcal{S}} q(s) \exp \left(\frac{-v(s)}{\lambda}\right)\right)$$

なので、$\mathcal{L}(\lambda)=-t \lambda-\lambda \log \left(q^T\exp \left(\frac{v}{\lambda}\right)\right)$
を最大化させる$\lambda \geq 0$で元の問題が解けます。

ちなみにこの問題を解く計算量は$\widetilde{\mathcal{O}}(|\mathcal{S}|)$になります。（
[Robust dynamic programming](http://www.corc.ieor.columbia.edu/reports/techreports/tr-2002-07.pdf)参照）

他の距離についても似たようなことができます。（[Robust dynamic programming](http://www.corc.ieor.columbia.edu/reports/techreports/tr-2002-07.pdf)参照）